# Ayudantía 05 - Árboles y Listas Ligadas
__Autores: Brian Murtagh ([@bmurtagh01](https://github.com/bmurtagh01)) e Ian Basly ([@igbasly](https://github.com/igbasly))__

## DCCableling

El malvado Doctor Valdivieso, cansado del exilio ha decidido robar todos los cables del sistema eléctrico de Chile para construir el gran Olvida-inador, un artefacto capaz de borrar cualquier leguaje de programación de la mente de las personas.

<img src="files/logo_pynel.png" alt="drawing" width="300"/>Es por esto, que Pynel ha elegido a los grandes estudiantes del IIC2233 para modelar la reconstrucción del sistema mediante el DCCableling, así poder restablecer el sistema el ́ectrico y hackear la malévola máquina para iniciar su autodestrucción.

### Sistemas eléctricos

Cada sistema eléctrico está modelado en forma de árboles, donde cada nodo es una instalación del sistema. Estás instalaciones se encuentran en el orden jerarquico: **Central generadora, Distribuidora regional, Distribuidora comunal, Casa.** Donde la Central generadora es el nodo raaíz del sistema.

<img src="files/arbol.png" alt="drawing" width="900"/>

Cada instalación posee los siguientes atributos:
- **`self.tipo`**: *string* que indica el tipo de instalción.**"Generadora", "Distribuidora Regional", "Distribuidora Comunal" o "Casa".**
- **`self.region`**: *string* con la región en la que se encuentra la instalación.
- **`self.comuna`:** *string* con la comuna en la que se encuentra la instalación.
- **`self.hijos`:** *list* para contener a los hijos que dependen directamente de ella.
- **`self.energia`**: *float* con la energía que posee la instalación al comienzo.
- **`self.consumo`:** *float* que representa la cantidad de energía utilizada por la instalción.


### Aramado del sistema

Para el sistema, lo primero que se crea es la **Central generadora**, quien se encaraga de recibir todas las nuevas instalaciones. Cada instalación se comporta de igual manera para recibir instalaciones, si la instalación que está recibiendo es del nivel jerarquico **directamente siguiente**, entonces la conecta como hija.
En caso que no se cumpla lo anterior, entonces debe elegir uno de sus hijos para entregarle la nueva instalación. Esta selección se hace en base a las caracteristicas geográficas de la instalación, en el caso de las Distribuidoras regionales buscarán una coincidencia en la **región**, en cambio, las Distribuidoras Comunales en la **comuna** de la nueva instalación. Así hasta que la nueva instalación llegue a su lugar.

Un ejemplo:
<img src="files/ejemplo.png" alt="drawing" width="900"/>

Al agregar esta nueva casa, se debe buscar primero la Distribuidora regional de su región y luego la Distribuidora comunal de su comuna, donde quedará finalmente ubicada.

### Métodos

In [ ]:
def agregar_instalación:(self, instalacion):

Recibe una instalación y la agrega a sus conexiones directas o escoge entre una de sus conexiones actuales a la más conveniente, para que esta a su vez repita el proceso.

In [ ]:
def distribuir_energia(self, energia_recibida):

Es un método que distribuye la energía recibida hacia cada uno de sus hijos siguiendo la siguiente formula:

- `E_hijo` = (energia_recibida - consumo) / numero_de_hijos

Entonces, para cada hijo, se le entregará `E_hijo` como `energia_recibida`.

## A trabajar!

In [ ]:
from textwrap import indent


class Instalacion:
    hijos = {
        "Generadora": "Distribuidora Regional",
        "Distribuidora Regional": "Distribuidora Comunal",
        "Distribuidora Comunal": "Casa",
        "Casa": None
    }

    def __init__(self, **kwargs):
        self.tipo = kwargs["tipo"]
        self.region = kwargs["region"]
        self.comuna = kwargs["comuna"]
        self.hijos = []
        self.consumo = float(kwargs["consumo"])
        self.energia = float(kwargs["energia"])

    def agregar_instalacion(self, instalacion):
        # Implementar
        """
        Agrega la instalación al sistema.
        """
        if instalacion.tipo == Instalacion.hijos[self.tipo]:
            """
            Si la instalacion a agregar es del tipo de nuestros hijos
            la agregamos
            """
            self.hijos.append(instalacion)
        else:
            if instalacion.tipo == "Distribuidora Comunal":
                """
                Si no es nuestro hijo y es comunal, buscamos la Regional
                adecuada para agregar.
                """
                for hijo in self.hijos:
                    if hijo.region == instalacion.region:
                        hijo.agregar_instalacion(instalacion)
                        break
            elif instalacion.tipo == "Casa" and self.tipo == "Generadora":
                """
                Si no es nuestro hijo y es Casa buscamos la Regional y
                Comunal adecuada para agregar.
                """
                for hijo in self.hijos:
                    if hijo.region == instalacion.region:
                        hijo.agregar_instalacion(instalacion)
                        break
            else:
                for hijo in self.hijos:
                    if hijo.comuna == instalacion.comuna:
                        hijo.agregar_instalacion(instalacion)
                        break

    def distribuir_energia(self, energia):
        cantidad_hijos = len(self.hijos)
        self.energia = energia
        for hijo in self.hijos:
            energia_hijo = (self.energia - self.consumo) / cantidad_hijos
            energia_hijo = max(0, energia_hijo)
            hijo.distribuir_energia(energia_hijo)
    
    def __repr__(self):
        repr_hijos = [indent(str(hijo), "    ") for hijo in self.hijos]
        
        salto = "\n" if repr_hijos else ""
            
        return f"{self.tipo}: {self.comuna} - {self.region}, energia: {round(self.energia, 2)}{salto}" +\
                "\n".join(repr_hijos)

In [ ]:
def contar_instalaciones(sistema, n=1):
    """
    Retornamos 1 y sumamos uno por cada uno de nuestros hijos
    """
    for hijo in sistema.hijos:
        n += contar_instalaciones(hijo)
    return n 

def contar_consumo(sistema):
    """
    Returnamos nuestro consumo más la suma del consumo de nuestros hijos
    """
    consumo = sistema.consumo
    for hijo in sistema.hijos:
        consumo += contar_consumo(hijo)
    return consumo

In [ ]:
from files.archivos import generator, instanciar_sistema
import os

# =========== NO MODIFICAR ===========
def resumen_sistema(sistema):
    print("Resumen del Sistema:")
    print(f"Consumo Total: {contar_consumo(sistema)}")
    print(f"Número de Instalaciones: {contar_instalaciones(sistema)}")
    
    print(sistema)

# Se probará DCCableling para 4 sistemas electricos:    
generador = generator()
for i in range(1, 5):
    print(f"CONSULTAS SISTEMA ELÉCTRICO N°{i}")
    print("---------------------" * 2)
    atributos_de_sistema = next(generador)
    sistema_electrico = instanciar_sistema(Instalacion, atributos_de_sistema)
    resumen_sistema(sistema_electrico)
    #comuna_mayor_gasto(sistema_electrico)
    #casas_insuficiencia(sistema_electrico)
    print("\n"+"---------------------" * 2)